In [2]:
import requests
from dotenv import load_dotenv
import os
import json

In [3]:
# prepare headers
load_dotenv()

token = os.getenv("TMDB_BEARER")

headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}

In [4]:
# test authentication
url = "https://api.themoviedb.org/3/authentication"
response = requests.get(url, headers=headers)
response.text

'{"success":true}'

In [7]:
# get popular series
url = "https://api.themoviedb.org/3/tv/popular"
response = requests.get(url, headers=headers)

popular_series = response.json()
popular_series.keys()

dict_keys(['page', 'results', 'total_pages', 'total_results'])

In [8]:
len(popular_series['results'])

20

In [10]:
popular_series['results'][1]

{'adult': False,
 'backdrop_path': '/9P4IIMYY3HifqeruZq0ZZ9g7YUi.jpg',
 'genre_ids': [10765, 9648, 10759],
 'id': 66732,
 'origin_country': ['US'],
 'original_language': 'en',
 'original_name': 'Stranger Things',
 'overview': 'When a young boy vanishes, a small town uncovers a mystery involving secret experiments, terrifying supernatural forces, and one strange little girl.',
 'popularity': 259.7541,
 'poster_path': '/cVxVGwHce6xnW8UaVUggaPXbmoE.jpg',
 'first_air_date': '2016-07-15',
 'name': 'Stranger Things',
 'vote_average': 8.594,
 'vote_count': 18978}

In [11]:
# search by series name (and get id)
url = "https://api.themoviedb.org/3/search/tv"
params = {"query": "Chicago Med"}
response = requests.get(url, headers=headers, params=params)
series_id = response.json()["results"][0]["id"]
series_id

62650

In [12]:
# get all genres and ids for the ones we want to exclude
url = "https://api.themoviedb.org/3/genre/tv/list"
response = requests.get(url, headers=headers)
result = response.json()

genre_to_id = {d["name"]: d["id"] for d in result["genres"]}
print(genre_to_id)

genres_to_exclude = [
    "Animation",
    "Family",
    "Kids",
    "News",
    "Reality",
    "Sci-Fi & Fantasy",
    "Soap",
    "Talk",
    "Western",
]
ids_to_exclude = ",".join([str(genre_to_id[genre]) for genre in genres_to_exclude])
ids_to_exclude

{'Action & Adventure': 10759, 'Animation': 16, 'Comedy': 35, 'Crime': 80, 'Documentary': 99, 'Drama': 18, 'Family': 10751, 'Kids': 10762, 'Mystery': 9648, 'News': 10763, 'Reality': 10764, 'Sci-Fi & Fantasy': 10765, 'Soap': 10766, 'Talk': 10767, 'War & Politics': 10768, 'Western': 37}


'16,10751,10762,10763,10764,10765,10766,10767,37'

In [13]:
# get all series with some filters
params = {
    "with_origin_country": "US",
    "first_air_date.gte": "2025-01-01",
    "sort_by": "popularity.desc",
    "without_genres": ids_to_exclude,
}

# get page 1 to get total pages
url = "https://api.themoviedb.org/3/discover/tv"
params["page"] = 1
response = requests.get(url, headers=headers, params=params)
data = response.json()

In [14]:
# get all pages
total_pages = data["total_pages"]

results_all = data["results"]
for page in range(2, total_pages + 1):
    url = "https://api.themoviedb.org/3/discover/tv"
    params["page"] = page
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    results_all.extend(data["results"])

In [15]:
len(results_all)

1037

In [17]:
for results_dict in results_all[:5]:
    print(results_dict["original_name"], results_dict["id"])

IT: Welcome to Derry 200875
The Last Frontier 221079
The Beast in Me 250504
Robin Hood 272418
Dinastía Casillas 302463


In [19]:
# get more details
url = f"https://api.themoviedb.org/3/tv/{series_id}"
params = {
    "append_to_response": "credits,keywords,recommendations,similar,content_ratings,videos,images,aggregate_credits,reviews"
}
response = requests.get(url, headers=headers, params=params)
details = response.json()
details.keys()

dict_keys(['adult', 'backdrop_path', 'created_by', 'episode_run_time', 'first_air_date', 'genres', 'homepage', 'id', 'in_production', 'languages', 'last_air_date', 'last_episode_to_air', 'name', 'next_episode_to_air', 'networks', 'number_of_episodes', 'number_of_seasons', 'origin_country', 'original_language', 'original_name', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'seasons', 'spoken_languages', 'status', 'tagline', 'type', 'vote_average', 'vote_count', 'credits', 'keywords', 'recommendations', 'similar', 'content_ratings', 'videos', 'images', 'aggregate_credits', 'reviews'])

In [12]:
# save to json to inspect
with open("../data/example_chicago_med_details.json", "w") as f:
    json.dump(details, f, indent=2)